In [45]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [37]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
menu_esempio = "../Hackapizza Dataset/Menu/Anima Cosmica.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [38]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [47]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

In [ ]:
def estrai_schema(testo, llm):
    
    # Prompt migliorato per il modello AI
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti e Tecniche. 
    
    Mi serve che analizzi il testo e identifichi le entità che meglio descrivono il contenuto (voglio che tu rimanga il più generale possibile).
    Un esempio di entità è Ristorante, un'altra Pianeta, un'altra Ingrediente. 

    Voglio che mi ritorni un JSON con:
    - La chiave come il nome della entità
    - Il valore float che mi indichi quanto è rilevante in percentuale l'entità nel testo in una scala da 1 a 100

    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.

    Testo:
    {testo}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        sostanze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle sostanze:", content)
        sostanze = {}

    return sostanze

In [ ]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo
entita_domande = estrai_schema(testo=testo_domande, llm=llm)
print("Entità CSV:", entita_domande)

Entità CSV: {'Piatto': 90.0, 'Ingrediente': 85.0, 'Tecnica': 80.0, 'Ristorante': 75.0, 'Pianeta': 70.0, 'Licenza': 60.0, 'Chef': 55.0, 'Codice': 40.0, 'Galassia': 35.0}


In [35]:
# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo
entita_pianeti = estrai_schema(testo=testo_pianeti, llm=llm)
print("Entità CSV:", entita_pianeti)

Entità CSV: {'Pianeta': 80, 'Valori numerici': 15, 'Tabella dati': 5}


In [ ]:
# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
entita_manuale = estrai_schema(testo=testo_manuale, llm=llm)
print("Entità PDF:", entita_manuale)

Entità PDF: {'Tecniche di cucina': 35.0, 'Ingredienti': 25.0, 'Ambiente spaziale': 15.0, 'Licenze': 10.0, 'Ordini': 10.0}


In [41]:
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
entita_codice = estrai_schema(testo=testo_codice, llm=llm)
print("Entità PDF:", entita_codice)

Entità PDF: {'Federazione Intergalattica': 15.0, 'Ordini': 25.0, 'Ingredienti': 20.0, 'Tecniche': 15.0, 'Licenze': 15.0, 'Sanzioni': 10.0}


In [42]:
# Esegui l'analisi del PDF
testo_menu = estrai_testo_pdf(menu_esempio)
entita_menu = estrai_schema(testo=testo_menu, llm=llm)
print("Entità PDF:", entita_menu)

Entità PDF: {'Ristorante': 95.0, 'Chef': 90.0, 'Pianeta': 85.0, 'Menu': 80.0, 'Piatto': 75.0, 'Ingrediente': 70.0, 'Tecnica Culinaria': 65.0, 'Licenze': 50.0}


In [48]:
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
entita_blog1 = estrai_schema(testo=testo_blog1, llm=llm)
print("Entità PDF:", entita_blog1)


Entità PDF: {'Ristorante': 25.0, 'Piatti': 20.0, 'Ingrediente': 20.0, 'Chef': 15.0, 'Tecnologia': 15.0, 'Pianeta': 5.0}


In [49]:
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)
entita_blog2 = estrai_schema(testo=testo_blog2, llm=llm)
print("Entità PDF:", entita_blog2)

Entità PDF: {'Ristorante': 25.0, 'Chef': 10.0, 'Piatto': 25.0, 'Ingrediente': 25.0, 'Tecnica': 10.0, 'Location': 5.0}


In [ ]:
def combina_punteggi(json_list, llm):
    """Combina i punteggi delle entità da più JSON, normalizzando sinonimi tramite LLM."""
    punteggi_totali = {}
    conteggio = {}
    
    for entita_manuale in json_list:
        for entita, punteggio in entita_manuale.items():
            if entita in punteggi_totali:
                punteggi_totali[entita] += punteggio
                conteggio[entita] += 1
            else:
                punteggi_totali[entita] = punteggio
                conteggio[entita] = 1
    
    punteggi_medi = {entita: punteggi_totali[entita] / conteggio[entita] for entita in punteggi_totali}
    
    # Normalizzazione con LLM
    prompt = f"""
    Data la seguente lista di entità con punteggi, raggruppa sinonimi e varianti sotto un'unica entità comune.
    Restituisci il risultato come JSON con le entità normalizzate e i punteggi medi aggregati.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    {json.dumps(punteggi_medi, indent=2, ensure_ascii=False)}
    """
    
    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        sostanze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle sostanze:", content)
        sostanze = {}

    return sostanze
    #entita_normalizzate = json.loads(response.strip())
    #return dict(sorted(entita_normalizzate.items(), key=lambda item: item[1], reverse=True))

In [68]:
json_list = [entita_codice, entita_domande, entita_manuale, entita_menu, entita_pianeti, entita_blog1, entita_blog2]
punteggi_combinati = combina_punteggi(json_list, llm)
print("Punteggi combinati:", punteggi_combinati)

Punteggi combinati: {'Ingrediente': 36.25, 'Piatto': 41.66666666666667, 'Tecnica': 40.0, 'Licenza': 42.5, 'Spazio': 31.25, 'Dati': 10.0, 'Ordini': 17.5, 'Sanzioni': 10.0, 'Ristorante': 55.0, 'Menu': 80.0, 'Tecnologia': 15.0, 'Location': 5.0, 'Chef': 42.5, 'Codice': 40.0}


In [71]:
# Filtra le entità con punteggio superiore a 20
entita_filtrate = [entita for entita, punteggio in punteggi_combinati.items() if punteggio > 15]

print(entita_filtrate)

['Ingrediente', 'Piatto', 'Tecnica', 'Licenza', 'Spazio', 'Ordini', 'Ristorante', 'Menu', 'Chef', 'Codice']


In [72]:
def estrai_valorizzazioni(testo, entita_filtrate, llm, driver):
    # Genera un prompt basato sulle entità filtrate
    entita_str = ', '.join(entita_filtrate)
    
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti e Tecniche. 
    Il testo che segue potrebbe contenere informazioni pertinenti a queste entità: {entita_str}
    Voglio che tu estragga possibili valorizzazioni di queste entità e me le ritorni in un JSON.

    Testo:
    {testo}
    """

    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle valorizzazioni:", content)
        valorizzazioni = {}
    
    return valorizzazioni


In [ ]:
valori_manuale = estrai_valorizzazioni(testo_manuale, entita_filtrate, llm, driver)
#valorizzazioni

Errore nel parsing JSON delle valorizzazioni: {
  "chef": [
    {
      "nome": "Sirius Cosmo",
      "descrizione": "Lo chef stellare per eccellenza, sinonimo di cucina galattica."
    }
  ],
  "licenza": [
    {
      "tipo": "Psionica",
      "livelli": [
        {
          "livello": 0,
          "descrizione": "Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti."
        },
        {
          "livello": "I",
          "descrizione": "Lettura del pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti."
        },
        {
          "livello": "II",
          "descrizione": "Manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell’universo."
        },
        {
          "livello": "III",
          "descrizione": "Capacità di donare coscienza e intelletto ad oggetti, manipolazione della realtà

In [74]:
valori_codice = estrai_valorizzazioni(testo_codice, entita_filtrate, llm, driver)

In [ ]:
    # Funzione per caricare i dati in Neo4j
    def carica_neo4j(valorizzazioni):
        with driver.session() as session:
            for entita, valore in valorizzazioni.items():
                session.run("""
                    MERGE (e:Entita {nome: $entita})
                    SET e.valore = $valore
                """, entita=entita, valore=valore)

    # Carica le valorizzazioni su Neo4j
    carica_neo4j(valorizzazioni)

## MAIN

### SOSTANZE

In [29]:
sostanze = estrai_sostanze(llm, file_path)

In [30]:
print(sostanze)

{'Erba Pipa': 'CRP: 0.89, IEI: 0.3', 'Cristalli di Memoria': 'CRP: 0.92, CDT: 0.2', 'Petali di Eco': 'CRP: 0.87, IPM: 0.6', 'Carne di Drago': 'IPM: 0.85, IBX: 0.75', 'Uova di Fenice': 'IPM: 0.98, CDT: 0.8', 'Lacrime di Unicorno': 'IPM: 0.95, θ: 0.9', 'Foglie di Mandragora': 'IBX: 0.82, μ: 0.3', 'MuNa Lunare': 'IBX: 0.78, μ: 0.6, CRP: 0.72', 'Nettare di Sirena': 'IBX: 0.85, μ: 0.2, θ: 0.7', 'Spore Quantiche': 'δQ: 0.45, ID: 0.8', 'Essenza di Vuoto': 'δQ: 0.16, ID: 0.95', 'Funghi dell’Etere': 'δQ: 0.38, ID: 0.75', 'Sale Temporale': 'CDT: 0.65, IEI: 0.4', 'Radici di Gravità': 'CDT: 0.55, ID: 0.4', 'Polvere di Stelle': 'CDT: 0.75, IPM: 0.88'}


In [ ]:
with driver.session() as session:
        for nome, proprieta in sostanze.items():
            session.execute_write(aggiungi_sostanza, nome, proprieta)

C:\Users\lparigi\AppData\Local\Temp\ipykernel_29264\3805961789.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(aggiungi_sostanza, nome, proprieta)


### CREA RELAZIONI TECNICA RICHIEDE LICENZA 

In [33]:
def estrai_testo_capitolo_4(pdf_path: str) -> str:
    """
    Estrae il testo della sezione 4 del PDF che contiene le descrizioni delle tecniche e licenze.

    Args:
        pdf_path (str): Percorso al file PDF.

    Returns:
        str: Il testo del capitolo 4.
    """
    testo_capitolo_4 = ""

    with pdfplumber.open(pdf_path) as doc:
        for page in doc.pages:
            testo = page.extract_text()
            if testo and "4." in testo:  # Cerca il capitolo 4
                testo_capitolo_4 += testo

    if not testo_capitolo_4:
        print("Sezione 4 non trovata nel PDF.")
    return testo_capitolo_4

testo = estrai_testo_capitolo_4(file_path)
print(testo)

789/12773 67° Giorno del Ciclo Cosmico 789
2. Ingredienti di Origine Mitica: sostanze derivate da creature leggendarie, la cui
esistenza trascende le normali categorie biologiche e richiede considerazioni sia
pratiche che etiche nella loro gestione.
3. Sostanze Xenobiologiche: materiali di origine biologica non terrestre che richiedono
particolare attenzione nella manipolazione e conservazione, con potenziali
interazioni con i sistemi biologici terrestri.
4. Sostanze Quantiche e Dimensionali: ingredienti che manifestano proprietà
quantistiche o multi-dimensionali, necessitando di specifiche precauzioni per
prevenire anomalie spazio-temporali.
5. Sostanze Spazio-Temporali: elementi che interagiscono con il tessuto spazio-
temporale, richiedendo particolare attenzione per evitare distorsioni locali della
realtà.
2.2 Tabella delle sostanze regolamentate
La presente tabella fornisce una panoramica delle principali categorie di sostanze
regolamentate.
Sostanza Proprietà principali
Sostanze 

In [ ]:
def estrai_tecniche_licenze(testo: str) -> list:
    """
    Estrae le tecniche e le licenze dal testo per la sezione del PDF riguardante la marinatura.

    Args:
        testo (str): Il testo contenente le tecniche e le licenze.

    Returns:
        list: Una lista di tuple contenenti il nome della tecnica e la licenza.
    """
    # Espressione regolare per estrarre la tecnica, la licenza e il livello
    pattern = r"\s*([A-Za-z\s]+(?:[a-zA-Z\s]+))\s*richiede\s*una\s*licenza\s*([A-Za-z]+)\s*\([a-zA-Z\+\-]+\)\s*di\s*livello\s*(I{1,3}|IV|[1-9]+)"

    tecniche_licenze = re.findall(pattern, testo)

    # Processa i risultati per formattare correttamente la tecnica e la licenza
    tecniche_licenze_formattate = []
    for tecnica, licenza, livello in tecniche_licenze:
        # Rimuovi l'articolo "La" dalla tecnica
        tecnica = tecnica.strip().replace("La ", "")
        
        # Capitalizza correttamente la tecnica (ogni parola inizia con una lettera maiuscola)
        tecnica = ' '.join([parola.capitalize() for parola in tecnica.split()])

        # Crea la licenza nel formato "Licenza livello X"
        licenza_formattata = f"{licenza.capitalize()} livello {livello}"

        tecniche_licenze_formattate.append((tecnica, licenza_formattata))

    return tecniche_licenze_formattate



tecniche_licenze = estrai_tecniche_licenze(testo)

In [45]:
print(tecniche_licenze)

[('Marinatura A Infusione Gravitazionale', 'Gravitazionale livello II'), ('Marinatura Temporale Sincronizzata', 'Temporale livello I'), ('Antimateria Diluite', 'Antimateria livello I'), ('Umicatura A Stratificazione Quantica', 'Quantistica livello 3'), ('Umicatura Psionica Sensoriale', 'Psionica livello II'), ('Umicatura Polarizzata A Freddo Iperbarico', 'Magnetica livello I'), ('Fermentazione Quantica A Strati Multiversali', 'Quantistica livello 5'), ('Fermentazione Psionica Energetica', 'Psionica livello I'), ('Fermentazione Quantico Biometrica', 'Quantistica livello 3'), ('Decostruzione Atomica A Strati Energetici', 'Antimaterica livello I'), ('Decostruzione Magnetica Risonante', 'Magnetica livello I'), ('Decostruzione Ancestrale', 'Temporale livello III'), ('Decostruzione Interdimensionale Lovercraftiana', 'Quantistica livello 7'), ('Psionica Variabile', 'Psionica livello I'), ('Sferificazione Filamentare A Molecole Vibrazionali', 'Magnetica livello I'), ('Sferificazione Cromatica 

In [46]:
def aggiungi_relazione_licenza(tx, tecnica: str, licenza: str):
    """
    Aggiunge una relazione RICHIEDE_LICENZA tra una tecnica e la licenza nel database Neo4j.

    Args:
        tx: Transazione Neo4j
        tecnica (str): Nome della tecnica
        licenza (str): Nome della licenza
    """
    query = """
    MATCH (t:Tecnica {nome: $tecnica}), (l:Licenza {nome: $licenza})
    MERGE (t)-[:RICHIEDE_LICENZA]->(l)
    """
    tx.run(query, tecnica=tecnica, licenza=licenza)

In [47]:
with driver.session() as session:
    for tecnica, licenza in tecniche_licenze:
        session.execute_write(aggiungi_relazione_licenza, tecnica, licenza)

### CREA PIANETI

In [ ]:
def carica_pianeti_e_distanze(file_csv: str):
    """
    Carica i pianeti come nodi e le distanze tra di loro come relazioni nel database Neo4j.
    
    Args:
        file_csv (str): Percorso del file CSV contenente le distanze tra i pianeti.
        uri (str): URI del database Neo4j.
        user (str): Nome utente per Neo4j.
        password (str): Password per Neo4j.
    """
    
    # Carica il file CSV usando pandas
    df = pd.read_csv(file_csv, index_col=0)
    
    with driver.session() as session:
        # Crea i nodi per i pianeti
        pianeti = df.columns.tolist()  # Estrae i nomi dei pianeti
        
        for pianeta in pianeti:
            # Crea un nodo per ciascun pianeta
            query = "MERGE (p:Pianeta {nome: $nome})"
            session.run(query, nome=pianeta)
        
        # Crea le relazioni DISTANZA tra i pianeti
        for i, pianeta1 in enumerate(pianeti):
            for j, pianeta2 in enumerate(pianeti):
                if i < j:  # Evita di creare relazioni duplicate (ad esempio tra Tatooine e Asgard e viceversa)
                    distanza = df.at[pianeta1, pianeta2]
                    # Crea una relazione con la distanza tra i pianeti
                    query = """
                    MATCH (p1:Pianeta {nome: $pianeta1}), (p2:Pianeta {nome: $pianeta2})
                    MERGE (p1)-[:DISTANZA {km: $distanza}]->(p2)
                    """
                    session.run(query, pianeta1=pianeta1, pianeta2=pianeta2, distanza=distanza)
        
        print("Pianeti e distanze caricate correttamente in Neo4j.")

# Esegui il codice passando il percorso del file CSV e le credenziali di Neo4j
file_csv = '../Hackapizza Dataset/Misc/Distanze.csv'  # Percorso al tuo file CSV

carica_pianeti_e_distanze(file_csv)


<>:39: SyntaxWarning: invalid escape sequence '\M'
<>:39: SyntaxWarning: invalid escape sequence '\M'
C:\Users\lparigi\AppData\Local\Temp\ipykernel_29264\567156097.py:39: SyntaxWarning: invalid escape sequence '\M'
  file_csv = '../Hackapizza Dataset\Misc\Distanze.csv'  # Percorso al tuo file CSV


Pianeti e distanze caricate correttamente in Neo4j.


In [54]:

def carica_piatti(file_json: str):
    """
    Carica i piatti e i numeri associati come nodi nel database Neo4j.
    
    Args:
        file_json (str): Percorso del file JSON contenente i piatti e i numeri.
        uri (str): URI del database Neo4j.
        user (str): Nome utente per Neo4j.
        password (str): Password per Neo4j.
    """
    
    # Carica il file JSON
    with open(file_json, 'r', encoding='utf-8') as f:
        piatti_data = json.load(f)
    
    with driver.session() as session:
        for piatto, numero in piatti_data.items():
            # Decodifica i caratteri speciali (ad esempio per trattare le sequenze \u00e0)
            piatto_decodificato = html.unescape(piatto)
            
            # Crea un nodo per ciascun piatto
            query = """
            MERGE (p:Piatto {nome: $nome})
            SET p.numero = $numero
            """
            session.run(query, nome=piatto_decodificato, numero=numero)
        
        print("Piatti caricati correttamente in Neo4j.")

# Esegui il codice passando il percorso del file JSON e le credenziali di Neo4j
file_json = '../Hackapizza Dataset/Misc/dish_mapping.json'  # Percorso al tuo file JSON

carica_piatti(file_json)


Piatti caricati correttamente in Neo4j.
